In [278]:
import numpy as np
import matplotlib.pylab as plt

In [20]:
import sympy as sp
from sympy.vector import CoordSys3D, BodyOrienter
from sympy import symbols

## Matrice de changement de base

In [8]:
omega, phi, psi = symbols('omega, phi, psi')

from https://docs.sympy.org/latest/_modules/sympy/vector/orienters.html

        A 'Body' fixed rotation is described by three angles and
        three body-fixed rotation axes. To orient a coordinate system D
        with respect to N, each sequential rotation is always about
        the orthogonal unit vectors fixed to D. For example, a '123'
        rotation will specify rotations about N.i, then D.j, then
        D.k. (Initially, D.i is same as N.i)

In [14]:
body_orienter = BodyOrienter(omega, psi, phi, 'YXZ')

In [64]:
R = body_orienter.rotation_matrix()
R

Matrix([
[sin(omega)*sin(phi)*sin(psi) + cos(omega)*cos(phi), sin(phi)*cos(psi), -sin(omega)*cos(phi) + sin(phi)*sin(psi)*cos(omega)],
[sin(omega)*sin(psi)*cos(phi) - sin(phi)*cos(omega), cos(phi)*cos(psi),  sin(omega)*sin(phi) + sin(psi)*cos(omega)*cos(phi)],
[                               sin(omega)*cos(psi),         -sin(psi),                                 cos(omega)*cos(psi)]])

In [80]:
code = sp.python(R)

In [81]:
print(code)

omega = Symbol('omega')
phi = Symbol('phi')
psi = Symbol('psi')
e = ImmutableDenseMatrix([[sin(omega)*sin(phi)*sin(psi) + cos(omega)*cos(phi), sin(phi)*cos(psi), -sin(omega)*cos(phi) + sin(phi)*sin(psi)*cos(omega)], [sin(omega)*sin(psi)*cos(phi) - sin(phi)*cos(omega), cos(phi)*cos(psi), sin(omega)*sin(phi) + sin(psi)*cos(omega)*cos(phi)], [sin(omega)*cos(psi), -sin(psi), cos(omega)*cos(psi)]])


In [82]:
M_text = "[[sin(omega)*sin(phi)*sin(psi) + cos(omega)*cos(phi), sin(phi)*cos(psi), -sin(omega)*cos(phi) + sin(phi)*sin(psi)*cos(omega)], [sin(omega)*sin(psi)*cos(phi) - sin(phi)*cos(omega), cos(phi)*cos(psi), sin(omega)*sin(phi) + sin(psi)*cos(omega)*cos(phi)], [sin(omega)*cos(psi), -sin(psi), cos(omega)*cos(psi)]]"

In [73]:
M_text = M_text.replace('sin', 's_').replace('cos', 'c_')
M_text = M_text.replace('(omega)', 'omega').replace('(phi)', 'phi').replace('(psi)', 'psi')
M_text = M_text.replace(', ', ',\n')
print(M_text)

[[s_omega*s_phi*s_psi + c_omega*c_phi,
s_phi*c_psi,
-s_omega*c_phi + s_phi*s_psi*c_omega],
[s_omega*s_psi*c_phi - s_phi*c_omega,
c_phi*c_psi,
s_omega*s_phi + s_psi*c_omega*c_phi],
[s_omega*c_psi,
-s_psi,
c_omega*c_psi]]


In [50]:
%%timeit
omega, psi, phi = np.pi/2, 0, 0

u = [np.sin(omega)*np.sin(phi)*np.sin(psi) + np.cos(omega)*np.cos(phi),
     np.sin(omega)*np.sin(psi)*np.cos(phi) - np.sin(phi)*np.cos(omega),
     np.sin(omega)*np.cos(psi)]
     
v = [ np.sin(phi)*np.cos(psi),
      np.cos(phi)*np.cos(psi),
     -np.sin(psi)]

w = [-np.sin(omega)*np.cos(phi) + np.sin(phi)*np.sin(psi)*np.cos(omega),
      np.sin(omega)*np.sin(phi) + np.sin(psi)*np.cos(omega)*np.cos(phi),
      np.cos(omega)*np.cos(psi)]

21.2 µs ± 531 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [85]:
%%timeit
omega, psi, phi = np.pi/2, 0, 1.1

s_omega, c_omega = np.sin(omega), np.cos(omega)
s_psi, c_psi = np.sin(psi), np.cos(psi)
s_phi, c_phi = np.sin(phi), np.cos(phi)

u = np.array([s_omega*s_phi*s_psi + c_omega*c_phi,
              s_omega*s_psi*c_phi - s_phi*c_omega,
              s_omega*c_psi])
     
v = np.array([ s_phi*c_psi,
               c_phi*c_psi,
              -s_psi])

w = np.array([-s_omega*c_phi + s_phi*s_psi*c_omega,
               s_omega*s_phi + s_psi*c_omega*c_phi,
               c_omega*c_psi])

u, v, w = (np.asarray(a, dtype=np.float64) for a in (u, v, w))

10.8 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Functions

In [280]:
def rotation_matrix(omega, psi, phi):
    '''Rotation matrix defined with Euler's angles
        i.e. Body Rotation around axis 'YXZ'
    '''
    s_omega, c_omega = np.sin(omega), np.cos(omega)
    s_psi, c_psi = np.sin(psi), np.cos(psi)
    s_phi, c_phi = np.sin(phi), np.cos(phi)

    R = np.array([[ s_omega*s_phi*s_psi + c_omega*c_phi,
                    s_phi*c_psi,
                   -s_omega*c_phi + s_phi*s_psi*c_omega],
                  [ s_omega*s_psi*c_phi - s_phi*c_omega,
                    c_phi*c_psi,
                    s_omega*s_phi + s_psi*c_omega*c_phi],
                  [ s_omega*c_psi,
                   -s_psi,
                    c_omega*c_psi]])
    
    return R

In [282]:
def change_base(A, u, angles, offset):
    omega, psi, phi = angles
    offset = np.asarray(offset)

    R = rotation_matrix(omega, psi, phi)

    A_prime = np.matmul(A, R.T) - offset
    u_prime = np.matmul(u, R.T)
    
    return A_prime, u_prime

# test
u = np.array([[-1, 0, 0], ])
A = np.array([[10, 0, 0], ])

change_base(A, u, (np.pi/2, 0, 0), (0, 0, 0))

(array([[6.123234e-16, 0.000000e+00, 1.000000e+01]]),
 array([[-6.123234e-17,  0.000000e+00, -1.000000e+00]]))